# Build a Chatbot

We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions with a chat model.

from [this](https://python.langchain.com/docs/tutorials/chatbot/) LangChain **v0.3** tutorial.

In [8]:
import langgraph
import langchain
import langchain_core
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

import os

from paper_query.constants.api_keys import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print(langchain.__version__)
print(langchain_core.__version__)

0.3.21
0.3.47


In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [4]:
model.invoke([HumanMessage(content="Hi! I'm Bob")])
# invoking the model is easy, but it currently has no concept of state i.e. it has no memory and cannot answer follow-up questions
# HumanMessage: HumanMessages are messages that are passed in from a human to the model.

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e4fa3702df', 'id': 'chatcmpl-BEEbdfOpQvGtB0LTEcFpTG7btW6z5', 'finish_reason': 'stop', 'logprobs': None}, id='run-dc4f33e1-62f8-4962-8f20-039be16a97ee-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
model.invoke([HumanMessage(content="What's my name?")])
# no concept of state / memory. It is STATELESS

AIMessage(content="I'm sorry, but I don't know your name. If you'd like to share it, feel free!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BEEbjrDKCnvmNzwWfVHjJyLdv4gCh', 'finish_reason': 'stop', 'logprobs': None}, id='run-43de1432-dc17-451d-ab44-7b37dc0f90ec-0', usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
# you can add multiple messages at once, which the model uses as "memory"
model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)
# For a chatbot we want the model to remeber previous messages and answers automatically

AIMessage(content='Your name is Bob! How can I assist you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 33, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BEEbrnT4Qla9qzMAWsfxMgg17Cz0E', 'finish_reason': 'stop', 'logprobs': None}, id='run-6de0ffeb-0c42-4a1f-8c8d-e823988699f3-0', usage_metadata={'input_tokens': 33, 'output_tokens': 15, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
# LangGraph has a built-in persistence layer i.e. it is STATEFUL.
# Wrapping out model in a minimal LangGraph app allows message history.

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define a new graph and the (single) node in the graph
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [13]:
# Different states are identified using a "thread_id". Different states are not aware of one another.
config = {"configurable": {"thread_id": "abc123"}}

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()
# "app" is STATEFUL and remebers previous messages

================================== Ai Message ==================================

Hello again, Bob! What’s on your mind today?
================================== Ai Message ==================================

Your name is Bob! What would you like to discuss?


In [ ]:
config = {"configurable": {"thread_id": "abc234"}}
# Change the thread_id to start a new conversation with a new history.
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't have access to personal information or previous interactions, so I don't know your name. If you'd like to share it or if there's something else you'd like to discuss, feel free to let me know!


In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a {job}. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [19]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    job: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages, "job": "pirate"}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! What be on ye mind today, matey?


In [25]:
from langchain_core.messages import trim_messages, SystemMessage

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [23]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "job": state["job"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [26]:
messages = [SystemMessage(content="You talk like a pirate. Answer all questions to the best of your ability.")]

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "job": "pirate"},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! I be glad t’ meet ye on this fine day! What be troublin’ yer sails, matey?


In [30]:
input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "job": "pirate"},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Ah|oy| there|,| Jim|!| What| brings| ye| to| these| tre|acher|ous| waters|?| Be| ye| look|in|’| for| treasure| or| seek|in|'| knowledge|,| mate|y|?| Arr|r|!||